In [5]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import cftime
#import cmaps
import cartopy.crs as ccrs

import intake
!conda install -c conda-forge xmip
from xmip.preprocessing import rename_cmip6

xr.set_options(display_style='html')
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 

ModuleNotFoundError: No module named 'xmip'

In [1]:
#!/usr/bin/env python
from __future__ import print_function
import requests
import xml.etree.ElementTree as ET
import numpy

# Author: Unknown
# I got the original version from a word document published by ESGF
# https://docs.google.com/document/d/1pxz1Kd3JHfFp8vR2JCVBfApbsHmbUQQstifhGNdc6U0/edit?usp=sharing

# API AT: https://github.com/ESGF/esgf.github.io/wiki/ESGF_Search_REST_API#results-pagination

def esgf_search(server="https://esgf-node.llnl.gov/esg-search/search",
                files_type="OPENDAP", local_node=True, project="CMIP6",
                verbose=False, format="application%2Fsolr%2Bjson",
                use_csrf=False, **search):
    client = requests.session()
    payload = search
    payload["project"] = project
    payload["type"]= "File"
    if local_node:
        payload["distrib"] = "false"
    if use_csrf:
        client.get(server)
        if 'csrftoken' in client.cookies:
            # Django 1.6 and up
            csrftoken = client.cookies['csrftoken']
        else:
            # older versions
            csrftoken = client.cookies['csrf']
        payload["csrfmiddlewaretoken"] = csrftoken

    payload["format"] = format

    offset = 0
    numFound = 10000
    all_files = []
    files_type = files_type.upper()
    while offset < numFound:
        payload["offset"] = offset
        url_keys = [] 
        for k in payload:
            url_keys += ["{}={}".format(k, payload[k])]

        url = "{}/?{}".format(server, "&".join(url_keys))
        print(url)
        r = client.get(url)
        r.raise_for_status()
        resp = r.json()["response"]
        numFound = int(resp["numFound"])
        resp = resp["docs"]
        offset += len(resp)
        for d in resp:
            if verbose:
                for k in d:
                    print("{}: {}".format(k,d[k]))
            url = d["url"]
            for f in d["url"]:
                sp = f.split("|")
                if sp[-1] == files_type:
                    all_files.append(sp[0].split(".html")[0])
    return sorted(all_files)

In [2]:
result = esgf_search(activity_id='OMIP', grid_label='gn', experiment_id='omip2', variable_id='so', nominal_resolution = '25 km')


#result = esgf_search(activity_id='OMIP', experiment_id='omip2', variable_id='so', source_id = 'ACCESS-OM2-025')

#result = esgf_search(activity_id='OMIP', grid_label='gn', experiment_id='omip2',
#                      variable_id='so', table_id='Omon', source_id = ['CMCC-CM2-HR4', 'ACCESS-OM2-025']) #institution_id = ['CMCC', 'CSIRO-COSIMA']) #, nominal_resolution = ['25 km']) #, source_id = ['CMCC-CM2-HR4', 'ACCESS-OM2-025'])
#                    #table_id=['Oday', 'SImon', 'Ofx']) 
len(result)

https://esgf-node.llnl.gov/esg-search/search/?activity_id=OMIP&grid_label=gn&experiment_id=omip2&variable_id=so&nominal_resolution=25 km&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=0
https://esgf-node.llnl.gov/esg-search/search/?activity_id=OMIP&grid_label=gn&experiment_id=omip2&variable_id=so&nominal_resolution=25 km&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=10
https://esgf-node.llnl.gov/esg-search/search/?activity_id=OMIP&grid_label=gn&experiment_id=omip2&variable_id=so&nominal_resolution=25 km&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=20
https://esgf-node.llnl.gov/esg-search/search/?activity_id=OMIP&grid_label=gn&experiment_id=omip2&variable_id=so&nominal_resolution=25 km&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=30
https://esgf-node.llnl.gov/esg-search/search/?activity_id=OMIP&grid_label=gn&experiment_id=omip2&variable_id=so&nominal_resolutio

475

In [3]:

#url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
#col = intake.open_esm_datastore(url)


### load a few models to illustrate the problem
#query = dict(activity_id='OMIP', grid_label='gn', experiment_id='omip2', variable_id='so', nominal_resolution = '25 km')
#cat = col.search(**query)

#cat.df['source_id'].unique()

## show coordinates
#for k, ds in dset_dict.items():
#    print(k)
#    print(list(ds.dims))

In [17]:
#files_to_open = result[-1]

In [6]:
ds = xr.open_mfdataset(result, combine='by_coords')
#ds = xr.open_mfdataset(files_to_open, combine='by_coords')
ds

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/coding/times.py:682: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)


TypeError: Cannot combine along dimension 'time' with mixed types. Found: DatetimeProlepticGregorian, DatetimeNoLeap. If importing data directly from a file then setting `use_cftime=True` may fix this issue.

In [ ]:
ds = rename_cmip6(ds)
ds